In [1]:
import os
import math
import numpy as np
import pandas as pd
import tensorflow as tf
from model import seq2seq
import gensim
from config import config


import os
import math
import numpy as np
import pandas as pd
import tensorflow as tf

from config import config
from dataset import create_dataset, load_embedding
from model import seq2seq


os.environ['CUDA_VISIBLE_DEVICES'] = '-1'         # specify GPU usage  

In [2]:
encoder = tf.keras.models.load_model(config.ENCODER_PATH)
encoder.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
encoder_inputs (InputLayer)  [(None, 100)]             0         
_________________________________________________________________
encoder_embedding (Embedding (None, 100, 300)          77976600  
_________________________________________________________________
encoder_lstm (LSTM)          [(None, 250), (None, 250) 551000    
Total params: 78,527,600
Trainable params: 551,000
Non-trainable params: 77,976,600
_________________________________________________________________


In [3]:
decoder = tf.keras.models.load_model(config.DECODER_PATH)
decoder.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
decoder_inputs (InputLayer)     [(None, 30)]         0                                            
__________________________________________________________________________________________________
decoder_embedding (Embedding)   (None, 30, 300)      77976600    decoder_inputs[0][0]             
__________________________________________________________________________________________________
decoder_state_input_h (InputLay [(None, 250)]        0                                            
__________________________________________________________________________________________________
decoder_state_input_c (InputLay [(None, 250)]        0                                            
____________________________________________________________________________________________

In [4]:
word2vec = load_embedding()

In [5]:
test_df = pd.read_csv(config.TEST_FILE) 

test_encoder_input, test_decoder_input, test_decoder_output = create_dataset(
    contexts=test_df[config.CONTENT_FIELD].values.astype('str'), 
    titles=test_df[config.TITLE_FIELD].values.astype('str'),
    word2vec=word2vec
)

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 1.061 seconds.
Prefix dict has been built successfully.


In [6]:
len(test_encoder_input)

100

In [15]:
k=1

print([word2vec.index_to_key[x] for x in test_encoder_input[k]][::-1][:100])
print([word2vec.index_to_key[x] for x in test_decoder_output[k]][:40])
print([word2vec.index_to_key[x] for x in test_decoder_input[k]][:40])

['据', '报道', '，', '伊朗', '表示', '愿意', '与', '世界大国', '进行', '新一轮', '的', '核', '谈判', '。', '伊朗核', '谈判代表', '写信给', '欧盟', '外交', '专员', '时', '表示', '，', '对', '5', '+', '1', '国家', '要', '回到', '对话', '的', '的', '政治', '意愿', '表示', '欢迎', '。', '他', '还', '说', '，', '双方', '应该', '保持', '接触', '，', '为', '新一轮', '的', '对话', '确定', '日期', '和', '地点', '。', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>']
['伊朗', '表示', '愿', '与', '世界大国', '约定', '核', '谈判', '日期', '和', '地点', '<END>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>']
['<START>', '伊朗', '表示', '愿', '与', '世界大国', '约定', 

In [12]:
states_value = encoder.predict(test_encoder_input)     # [state h, state_c]
target_seq = np.zeros((config.TEST_SIZE, config.MAX_LEN_DECODER))
target_seq[:,0] = word2vec.key_to_index['<START>']

# stop_condition = False
decoded_sentence = ""
decoded_sentences = [decoded_sentence]*config.TEST_SIZE


for i in range(config.MAX_LEN_DECODER):
    # while not stop_condition:
    output_tokens, h, c = decoder.predict([target_seq] + states_value)

    # Sample a token
    sampled_token_index = np.argmax(output_tokens, axis=-1)[:,0]
#     print(sampled_token_index)

    sampled_char = [word2vec.index_to_key[x] for x in sampled_token_index]
#     print(sampled_char)

    decoded_sentences_new=[]
    for i, decoded_sent in enumerate(decoded_sentences):
        decoded_sent += sampled_char[i]
        decoded_sentences_new.append(decoded_sent+' ')
    decoded_sentences = decoded_sentences_new

    # Update the target sequence (of length 1).
    target_seq = np.zeros((config.TEST_SIZE,config.MAX_LEN_DECODER))
    target_seq[:,0] = word2vec.key_to_index['<START>']

    # Update states
    states_value = [h, c]
    
    
decoded_sentences_cleaned = []
for i, sample in enumerate(decoded_sentences):
    position = sample.find('<EOS>')
    decoded_sentences_cleaned.append(sample[:position])
    

['中国 年终 年终 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜',
 '日本 畅游 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜',
 '“ 谢娜 年终 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜',
 '： 初冬 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜',
 '河南 谢娜 煮熟 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜',
 '北京 初冬 煮熟 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜',
 '“ 初冬 煮熟 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜',
 '男子 谢娜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜',
 '“ 谢娜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜',
 '美国 初冬 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜',
 '日本 谢娜 年终 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜',
 '“ 初冬 煮熟 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜',
 '男子 谢娜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜',
 '中国 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜',
 '： 初冬 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜 姜',
 '男子 谢娜 姜 

In [39]:
y_pred = [x.split(' ') for x in decoded_sentences_cleaned]

['中国',
 '年终',
 '年终',
 '姜',
 '姜',
 '姜',
 '姜',
 '姜',
 '姜',
 '姜',
 '姜',
 '姜',
 '姜',
 '姜',
 '姜',
 '姜',
 '姜',
 '姜',
 '姜',
 '姜',
 '姜',
 '姜',
 '姜',
 '姜',
 '姜',
 '姜',
 '姜',
 '姜',
 '姜',
 '姜']

In [37]:
y_true = []

for i, sample in enumerate(test_decoder_output):
    sent = [word2vec.index_to_key[x] for x in sample if word2vec.index_to_key[x] not in ['<PAD>','<END>']]
    y_true.append(sent)


In [47]:
from nltk.translate.bleu_score import corpus_bleu
bleu = corpus_bleu([[x] for x in y_true], y_pred)
bleu

/home/fengyuan/anaconda3/envs/transformers/lib/python3.6/site-packages/nltk/translate/bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/fengyuan/anaconda3/envs/transformers/lib/python3.6/site-packages/nltk/translate/bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/fengyuan/anaconda3/envs/transformers/lib/python3.6/site-packages/nltk/translate/bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower orde

3.2397263153811994e-232

In [52]:
from rouge import Rouge 
rouge = Rouge()
scores = rouge.get_scores([' '.join(x) for x in y_pred], [' '.join(x) for x in y_true], avg=True)

In [54]:
print(scores)

{'rouge-1': {'r': 0.018857004107004104, 'p': 0.05416666666666666, 'f': 0.02733727110433932}, 'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-l': {'r': 0.018857004107004104, 'p': 0.05416666666666666, 'f': 0.02733727110433932}}
